# Social Laws Use Case Demo

This notebook shows a demonstration of a realistic use case for social laws. In this demo we will synthesize a social law for a multi-agent pickup and delivery problem.


The maps will be specified as a list of strings where 
* \* denotes an obstacle
* a lower case letter (a-z) denotes the initial position of an agent
* an upper case letter (A-Z) denotes the goal position of the corresponding agent



In [6]:
map = [
"*******",
"* B A *",
"*  *  *",
"* a b *",
"*******"
]


We now define a function which converts the given map into a multi agent planning problem

In [7]:
from unified_planning.shortcuts import *
from unified_planning.social_law.ma_problem_waitfor import MultiAgentProblemWithWaitfor
from unified_planning.model.multi_agent import *
import string

def mapToMaProblem(map : str):
    problem = MultiAgentProblemWithWaitfor("MAPD")

    loc = UserType("loc")
    
    # Environment     
    connected = Fluent('connected', BoolType(), l1=loc, l2=loc)        
    problem.ma_environment.add_fluent(connected, default_initial_value=False)

    free = Fluent('free', BoolType(), l=loc)
    problem.ma_environment.add_fluent(free, default_initial_value=True)




    # Agents
    at = Fluent('at', BoolType(), l1=loc)

    move = InstantaneousAction('move', l1=loc, l2=loc)
    l1 = move.parameter('l1')
    l2 = move.parameter('l2')
    move.add_precondition(at(l1))
    move.add_precondition(free(l2))
    move.add_precondition(connected(l1,l2))
    move.add_effect(at(l2),True)
    move.add_effect(free(l2), False)
    move.add_effect(at(l1), False)
    move.add_effect(free(l1), True)    


    locmap = {}

    for i,line in enumerate(map):
        for j,cell in enumerate(line):
            locname = "l_" + str(i) + "_" + str(j)
            locobj = Object(locname, loc)
            problem.add_object(locobj)
            locmap[i,j] = locobj
            if cell == "*":
                problem.set_initial_value(free(locobj), False)
            elif cell in string.ascii_lowercase:
                agentname = "a_" + cell
                agent = Agent(agentname, problem)
                problem.add_agent(agent)
                agent.add_fluent(at, default_initial_value=False)
                agent.add_action(move)
                problem.set_initial_value(Dot(agent, at(locobj)), True)
    
    for i,line in enumerate(map):
        for j,cell in enumerate(line):
            if cell in string.ascii_uppercase:
                agentname = "a_" + cell.lower()
                locobj = locmap[i,j]
                agent = problem.agent(agentname)
                problem.add_goal(Dot(agent, at(locobj)))


    for i,j in locmap.keys():
        for dx,dy in [(-1,0),(+1,0),(0,-1),(0,+1)]:
            if (i+dx, j+dy) in locmap.keys():        
                problem.set_initial_value(connected(locmap[i,j], locmap[i+dx,j+dy]), True)        

    return problem


problem = mapToMaProblem(map)
print(problem)



problem name = MAPD

types = [loc]

environment fluents = [
  bool connected[l1=loc, l2=loc]
  bool free[l=loc]
]

agents = [
  Agent name = a_a

private fluents = [
 bool at[l1=loc]
]

public fluents = [
]

actions = [
 action move(loc l1, loc l2) {
    preconditions = [
      at(l1)
      free(l2)
      connected(l1, l2)
    ]
    effects = [
      at(l2) := true
      free(l2) := false
      at(l1) := false
      free(l1) := true
    ]
  }
]


  Agent name = a_b

private fluents = [
 bool at[l1=loc]
]

public fluents = [
]

actions = [
 action move(loc l1, loc l2) {
    preconditions = [
      at(l1)
      free(l2)
      connected(l1, l2)
    ]
    effects = [
      at(l2) := true
      free(l2) := false
      at(l1) := false
      free(l1) := true
    ]
  }
]


]

objects = [
  loc: [l_0_0, l_0_1, l_0_2, l_0_3, l_0_4, l_0_5, l_0_6, l_1_0, l_1_1, l_1_2, l_1_3, l_1_4, l_1_5, l_1_6, l_2_0, l_2_1, l_2_2, l_2_3, l_2_4, l_2_5, l_2_6, l_3_0, l_3_1, l_3_2, l_3_3, l_3_4, l_3_5, l_3_6, l_4_0

## We can now synthesize a social law for the map

In [8]:
from unified_planning.social_law.synthesis import get_gbfs_social_law_generator

generator = get_gbfs_social_law_generator()
robust_sl = generator.generate_social_law(problem)

print(robust_sl)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 128 of `/usr/local/lib/python3.10/dist-packages/unified_planning-0.6.0-py3.10.egg/unified_planning/social_law/robustness_checker.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 128 of `/usr/local/lib/python3.10/dist-packages/unified_planning-0.6.0-py3.10.egg/unified_planning/social_law/robustness_checker.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

  *** Credits ***
  * In operation mode `Compiler` at line 135

UPValueError: Agent a is not defined!

In [ ]:
large_map = [
"**************",
"* B        A *",
"*  *******   *",
"* a        b *",
"**************"
]
